In [3]:
# Cell 1: Imports and Setup (Ensure you have dtaidistance, simplification, scikit-learn, and torch installed)
import pandas as pd
from sklearn.cluster import DBSCAN
import joblib
import numpy as np
# Import dtw.fast for optimized DTW calculation
from dtaidistance import dtw
from simplification.cutil import simplify_coords
from sklearn.metrics import silhouette_score
from tqdm.notebook import tqdm # For progress bars in notebook

### Loading the data for one type of trip

In [4]:
# --- Configuration for Anomaly Detection (from original notebook) ---
NUM_EPOCHS = 10
features = ['latitude', 'longitude']
SEQ_LENGTH = 20
BATCH_SIZE = 32

In [5]:
df = pd.read_parquet("../data/cleaned/bremerhaven_anomalies_partlabeled.parquet")
df["time_stamp"] = pd.to_datetime(df["time_stamp"])
df = df.sort_values(by=["time_stamp", "trip_id"])

df = df[df["start_port"] == "BREMERHAVEN"]
print(df.value_counts("trip_id"))


trip_id
2035389    700
2035337    698
365365     698
1288934    695
1288936    695
          ... 
119446     170
688639     167
1173817    156
37617      146
119576     137
Name: count, Length: 703, dtype: int64


# Clustering with DBSCAN and DTW

We comperess the trajectories to reduce the number of points per trajectory.
As later we will use DTW to calculate the distance between trajectories, this will speed up the process.

In [6]:
# Clustering with DBSCAN and DTW - Trajectory Compression
# We compress the trajectories to reduce the number of points per trajectory.
# As later we will use DTW to calculate the distance between trajectories, this will speed up the process.
# Note: A more advanced Adaptive Douglas-Peucker (AF-DP) algorithm as described
# in the paper would involve dynamic epsilon selection and feature point retention
# based on movement characteristics (e.g., changes in speed/course).
# For this update, we keep `simplify_coords` but acknowledge the adaptive potential.

# Group Data into Trajectories
trajectories = {trip_id: group for trip_id, group in df.groupby('trip_id')}
print(f"Found {len(trajectories)} unique trajectories.")

# Compress Trajectories using the 'simplification' library
print("\nCompressing trajectories...")
compressed_trajectories = {}
# Using a general epsilon. An AF-DP implementation would dynamically set this
# based on trajectory characteristics for better feature preservation.
DEFAULT_DP_EPSILON = 0.0000001
for trip_id, traj_df in tqdm(trajectories.items(), desc="Compressing trajectories"):
    points = traj_df[['latitude', 'longitude']].values
    # Here, you would integrate the adaptive threshold logic from the paper
    # to dynamically determine 'epsilon' for each trajectory, and also
    # identify and retain "feature points" (e.g., sharp turns).
    # For simplicity, we use a fixed epsilon for now.
    compressed_points = simplify_coords(points, epsilon=DEFAULT_DP_EPSILON)
    compressed_trajectories[trip_id] = compressed_points

print("Trajectory compression complete.")
joblib.dump(compressed_trajectories, "data/compressed_trajectories.pkl")

Found 703 unique trajectories.

Compressing trajectories...


Compressing trajectories:   0%|          | 0/703 [00:00<?, ?it/s]

Trajectory compression complete.


['data/compressed_trajectories.pkl']

In [7]:
compressed_trajectories = joblib.load("data/compressed_trajectories.pkl")

In [8]:

# Cell 5: Calculate Distance Matrix with Fast-DTW
# This section replaces the standard DTW distance calculation with Fast-DTW.
# Fast-DTW offers significant speed improvements while maintaining accuracy.
print("\n--- Calculating Pairwise Distances using DTW ---")
trajectories_list = list(compressed_trajectories.values())

# Use dtw.distance_matrix for the optimized DTW algorithm (Corrected call)
# The 'compact' parameter returns a condensed distance matrix, saving memory
print("Calculating pairwise distances using DTW...")
distance_matrix = dtw.distance_matrix(trajectories_list, use_c=True, show_progress=True) # Corrected function call
print("DTW distance matrix calculation complete.")
joblib.dump(distance_matrix, "data/dtw_distance_matrix.pkl")


--- Calculating Pairwise Distances using DTW ---
Calculating pairwise distances using DTW...
DTW distance matrix calculation complete.


['data/dtw_distance_matrix.pkl']

In [9]:
distance_matrix = joblib.load("data/dtw_distance_matrix.pkl")
print("Loaded DTW distance matrix.")

Loaded DTW distance matrix.


In [10]:

# Cell 7: Adaptive DBSCAN Parameter Selection and Clustering
# This section implements the adaptive parameter selection using Silhouette Score
# as described in Section 2.5.2 of the paper.

print("\n--- Performing Adaptive DBSCAN Parameter Selection ---")

# Step 1: Generate candidate eps values
# The paper mentions using KANN and similarity distribution.
# A practical approach is to explore a range of percentiles of the distances.
# We'll use a range of percentiles from the distance matrix for epsilon candidates.
# Exclude diagonal (distance to self is 0) and flatten the upper triangle.
flat_distances = distance_matrix[np.triu_indices(distance_matrix.shape[0], k=1)].flatten()

# Ensure there are enough unique distances to compute percentiles
if len(np.unique(flat_distances)) > 10:
    eps_candidates = np.percentile(flat_distances, np.arange(5, 50, 5)) # From 5th to 45th percentile
    eps_candidates = np.unique(eps_candidates) # Ensure unique values
else:
    print("Warning: Not enough unique distances for percentile-based eps candidates. Using a default range.")
    eps_candidates = np.linspace(flat_distances.min() + 1e-6, flat_distances.max() / 2, 10) # Fallback range
    eps_candidates = eps_candidates[eps_candidates > 0] # Ensure eps > 0

# Step 2: Generate MinPts (min_samples) candidates
# The paper suggests "5 to eps list length". Let's use a reasonable range.
min_samples_candidates = list(range(3, 25, 2)) # e.g., 3, 5, 7, ..., 23

best_score = -1
best_params = {'eps': None, 'min_samples': None}
best_cluster_labels = None
results = [] # To store all tried combinations and their scores

print(f"Trying {len(eps_candidates)} epsilon candidates and {len(min_samples_candidates)} min_samples candidates.")
print(f"Epsilon candidates: {np.round(eps_candidates, 2)}")
print(f"Min_samples candidates: {min_samples_candidates}")


--- Performing Adaptive DBSCAN Parameter Selection ---
Trying 9 epsilon candidates and 11 min_samples candidates.
Epsilon candidates: [45.04 45.32 63.64 63.82 64.02 78.09 78.27 90.17 90.37]
Min_samples candidates: [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23]


In [11]:

# Perform grid search
for eps in tqdm(eps_candidates, desc="Searching eps"):
    for min_samples in min_samples_candidates:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
        cluster_labels = dbscan.fit_predict(distance_matrix)

        n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        n_noise = list(cluster_labels).count(-1)

        # Only calculate silhouette score if there are more than 1 cluster (excluding noise)
        # and not all points are noise.
        if 1 < n_clusters <= len(np.unique(cluster_labels[cluster_labels != -1])):
            # Filter out noise points for silhouette score calculation
            core_samples_mask = np.zeros_like(cluster_labels, dtype=bool)
            core_samples_mask[dbscan.core_sample_indices_] = True

            # Get indices of points that are part of a cluster (not noise)
            clustered_points_indices = np.where(cluster_labels != -1)[0]
            if len(clustered_points_indices) > 1:
                # Need distance matrix subset for silhouette calculation
                # Using distance_matrix (symmetric, so row/col indexing works for subset)
                sub_distance_matrix = distance_matrix[clustered_points_indices[:, None], clustered_points_indices]
                current_score = silhouette_score(sub_distance_matrix, cluster_labels[clustered_points_indices], metric='precomputed')
            else:
                current_score = -1 # Cannot calculate score for 0 or 1 clustered point
        else:
            current_score = -1 # Invalid number of clusters for silhouette score

        results.append({
            'eps': eps,
            'min_samples': min_samples,
            'num_clusters': n_clusters,
            'num_noise': n_noise,
            'silhouette_score': current_score
        })

        if current_score > best_score:
            best_score = current_score
            best_params['eps'] = eps
            best_params['min_samples'] = min_samples
            best_cluster_labels = cluster_labels

Searching eps:   0%|          | 0/9 [00:00<?, ?it/s]

In [12]:

print("\n--- Best DBSCAN Parameters Found ---")
if best_params['eps'] is not None:
    print(f"Optimal eps: {best_params['eps']:.4f}")
    print(f"Optimal min_samples: {best_params['min_samples']}")
    print(f"Best Silhouette Score: {best_score:.4f}")
else:
    print("Could not find optimal parameters. Defaulting to a fixed setting (eps=5.0, min_samples=5).")



--- Best DBSCAN Parameters Found ---
Optimal eps: 90.1681
Optimal min_samples: 15
Best Silhouette Score: 0.5700


In [37]:
# eps = best_params.get('eps', 5.0)  # Fallback to default if not found
# min_samples = best_params.get('min_samples', 5)  # Fallback to default if not found

eps = 0.8
min_samples = 5  # Default value if no optimal found

dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
cluster_labels = dbscan.fit_predict(distance_matrix)

In [38]:

# Identify the "normal" cluster (the largest one, excluding noise)
trip_ids = list(compressed_trajectories.keys())
# Filter out noise (-1) before finding the mode
cluster_labels_no_noise = cluster_labels[cluster_labels != -1]
normal_cluster_id_series = pd.Series(cluster_labels_no_noise).mode()

if not normal_cluster_id_series.empty:
    normal_cluster_id = normal_cluster_id_series[0]
    normal_trip_ids = [trip_id for trip_id, label in zip(trip_ids, cluster_labels) if label == normal_cluster_id]
    print(f"\nIdentified normal traffic pattern as Cluster ID: {normal_cluster_id}")
    print(f"Number of normal trips: {len(normal_trip_ids)}")
else:
    normal_trip_ids = []
    print("\nNo significant non-noise clusters found. Cannot proceed with normal cluster identification.")

# Print overall cluster distribution
unique_labels, counts = np.unique(cluster_labels, return_counts=True)
print("\nCluster distribution:")
for label, count in zip(unique_labels, counts):
    if label == -1:
        print(f"  Noise points (-1): {count} trajectories")
    else:
        print(f"  Cluster {label}: {count} trajectories")




Identified normal traffic pattern as Cluster ID: 2
Number of normal trips: 40

Cluster distribution:
  Noise points (-1): 166 trajectories
  Cluster 0: 31 trajectories
  Cluster 1: 36 trajectories
  Cluster 2: 40 trajectories
  Cluster 3: 16 trajectories
  Cluster 4: 19 trajectories
  Cluster 5: 15 trajectories
  Cluster 6: 6 trajectories
  Cluster 7: 30 trajectories
  Cluster 8: 15 trajectories
  Cluster 9: 6 trajectories
  Cluster 10: 28 trajectories
  Cluster 11: 27 trajectories
  Cluster 12: 27 trajectories
  Cluster 13: 34 trajectories
  Cluster 14: 30 trajectories
  Cluster 15: 17 trajectories
  Cluster 16: 8 trajectories
  Cluster 17: 21 trajectories
  Cluster 18: 30 trajectories
  Cluster 19: 16 trajectories
  Cluster 20: 24 trajectories
  Cluster 21: 21 trajectories
  Cluster 22: 7 trajectories
  Cluster 23: 8 trajectories
  Cluster 24: 7 trajectories
  Cluster 25: 7 trajectories
  Cluster 26: 6 trajectories
  Cluster 27: 5 trajectories


In [39]:
# Cell 8: Plotting the clusters (Keep as is, will use new cluster_labels)
from utils import visualize_trajectory_clusters_interactive_fancy

fig = visualize_trajectory_clusters_interactive_fancy(compressed_trajectories, trip_ids, cluster_labels)
fig.write_html("data/trajectories_clusters_improved.html", include_plotlyjs='cdn')
print("Interactive cluster visualization saved to data/trajectories_clusters_improved.html")


--- Optimized Trajectory Clusters Visualization ---
--- Visualization complete. Use the interactive map to explore clusters.
Interactive cluster visualization saved to data/trajectories_clusters_improved.html
